In [7]:
import pickle

with open('data/pickle/new.pickle', 'rb') as file:
    dict_data = pickle.load(file)

with open('data/pickle/new_pickle.txt', 'w') as f:
    f.write(str(dict_data)) 

In [9]:
# 한 글자 key 삭제
dict_data = {key: value for key, value in dict_data.items() if len(key) > 1}

with open('data/pickle/new.pickle', 'wb') as file:
    pickle.dump(dict_data, file)

with open('data/pickle/new.pickle', 'rb') as file:
    dict_data = pickle.load(file)

with open('data/pickle/new_pickle.txt', 'w') as f:
    f.write(str(dict_data))  

In [10]:
# key 제거
key_to_remove = '틀림'
if key_to_remove in dict_data:
    del dict_data[key_to_remove]

with open('data/pickle/new.pickle', 'wb') as file:
    pickle.dump(dict_data, file)

with open('data/pickle/new.pickle', 'rb') as file:
    dict_data = pickle.load(file)

with open('data/pickle/new_pickle.txt', 'w') as f:
    f.write(str(dict_data))  

In [ ]:
# value 제거
key = '시간'
value_to_remove = '성상'

if key in dict_data:
    dict_data[key] = [v for v in dict_data[key] if v != value_to_remove]

with open('data/pickle/new.pickle', 'wb') as file:
    pickle.dump(dict_data, file)

with open('data/pickle/new.pickle', 'rb') as file:
    dict_data = pickle.load(file)

with open('data/pickle/new_pickle.txt', 'w') as f:
    f.write(str(dict_data))  

In [11]:
# key 추가
dict_data['일단'] = ['우선, 먼저']

with open('data/pickle/new.pickle', 'wb') as file:
    pickle.dump(dict_data, file)

with open('data/pickle/new.pickle', 'rb') as file:
    dict_data = pickle.load(file)

with open('data/pickle/new_pickle.txt', 'w') as f:
    f.write(str(dict_data))  

In [22]:
import pandas as pd
import random

with open('data/pickle/new.pickle', 'rb') as file:
    dict_data = pickle.load(file)

def augment_sentence(sentence, dict_data):
    if not isinstance(sentence, str):
        return []
    
    words = sentence.split()
    new_sentences = []
    sentence_changed = False

    for i, word in enumerate(words):
        if word in dict_data:
            synonyms = [syn for syn in dict_data[word] if syn != word]
            if synonyms:
                sentence_changed = True
                for synonym in synonyms:
                    new_sentence = words[:i] + [synonym] + words[i+1:]
                    new_sentences.append(' '.join(new_sentence))
    
    if sentence_changed:
        return new_sentences
    else:
        return []

def generate_augmented_sentences(df, dict_data, labels, num_aug_per_label):
    augmented_rows = []
    test_rows = []

    for label, num_augmentations in zip(labels, num_aug_per_label):
        label_df = df[df['label'] == label]

        for _, row in label_df.iterrows():
            for column in ['sentence_1', 'sentence_2']:
                original_sentence = row[column]
                new_sentences = augment_sentence(original_sentence, dict_data)

                if new_sentences:
                    for new_sentence in new_sentences:
                        augmented_row = row.copy()
                        augmented_row[column] = new_sentence
                        augmented_row['source'] = f"{row['source']}-sr" 
                        augmented_rows.append(augmented_row)
                        
                        test_rows.append({
                            'original_sentence': original_sentence,
                            'augmented_sentence': new_sentence
                        })

                if len(augmented_rows) >= num_augmentations:
                    break
            
            if len(augmented_rows) >= num_augmentations:
                break

        while len(augmented_rows) < num_augmentations:
            random_row = label_df.sample(1).iloc[0]
            random_column = random.choice(['sentence_1', 'sentence_2'])
            random_sentence = random_row[random_column]
            new_sentences = augment_sentence(random_sentence, dict_data)

            if new_sentences:
                for new_sentence in new_sentences:
                    augmented_row = random_row.copy()
                    augmented_row[random_column] = new_sentence
                    augmented_row['source'] = f"{random_row['source']}-sr" 
                    augmented_rows.append(augmented_row)
                    
                    test_rows.append({
                        'original_sentence': random_sentence,
                        'augmented_sentence': new_sentence
                    })

    return pd.DataFrame(augmented_rows), pd.DataFrame(test_rows)

In [23]:
#####################################################
input_file = 'data/train.csv'  # 원본 파일 경로
test_file = 'data/sr_test.csv'  # 테스트 파일 경로
output_file = 'data/train_sr.csv'  # 저장 파일 경로

labels = [1.5, 2.5, 4.5]  # 증강이 필요한 라벨
num_aug_per_label = [504, 512, 798]  # 필요한 증강 개수
######################################################

In [24]:
df = pd.read_csv(input_file)

augmented_df, test_df = generate_augmented_sentences(df, dict_data, labels, num_aug_per_label)

augmented_df.to_csv(output_file, index=False)
test_df.to_csv(test_file, index=False)